In [1]:
import pandas as pd
import numpy as np
import re
import os

In [2]:
# path_to_mimic should link to the directory containing hosp, icu, ...
# the study is based on version 3.0
path_to_mimic = ".....physionet.org/files/mimiciv/3.0/"

# path_to_mimic_ed should link to the directory containing ed, ...
# the study is based on version 2.2
path_to_mimic_ed = ".....physionet.org/files/mimic-iv-ed/2.2/"

# path_to_mimic_ecg should link to the directory containing record_list.csv
# the study is based on version 1.0
path_to_mimic_ecg = ".....physionet.org/files/"

# path to local files
# should contain revasc_d_icd_procedures.xlsx, cabg_d_icd_procedures.xlsx, ...
path_to_local_files = "../data/"

In [3]:
admissions = pd.read_csv(os.path.join(path_to_mimic, 'hosp/admissions.csv'))

In [4]:
admissions.shape

(546028, 16)

In [5]:
admissions.columns

Index(['subject_id', 'hadm_id', 'admittime', 'dischtime', 'deathtime',
       'admission_type', 'admit_provider_id', 'admission_location',
       'discharge_location', 'insurance', 'language', 'marital_status', 'race',
       'edregtime', 'edouttime', 'hospital_expire_flag'],
      dtype='object')

In [6]:
admissions['subject_id'].nunique()

223452

In [7]:
filtered_admissions = admissions[admissions['edregtime'].notna()]
filtered_admissions.shape

(379240, 16)

In [8]:
filtered_admissions['subject_id'].nunique()

182439

In [9]:
icu_stays = pd.read_csv(os.path.join(path_to_mimic, 'icu/icustays.csv'))

In [10]:
icu_stays.shape

(94458, 8)

In [11]:
# Check if all hadm_id in icu_stays are present in admissions
all_ids_present = icu_stays['hadm_id'].isin(admissions['hadm_id']).all()
print("All hadm_id in icu_stays are present in admissions:", all_ids_present)

All hadm_id in icu_stays are present in admissions: True


In [12]:
ed_stays = pd.read_csv(os.path.join(path_to_mimic_ed, 'ed/edstays.csv'))

In [13]:
ed_stays.shape

(425087, 9)

In [14]:
ed_stays['subject_id'].nunique()

205504

In [15]:
ed_stays['disposition'].value_counts()

disposition
HOME                           241632
ADMITTED                       158010
TRANSFER                         7025
LEFT WITHOUT BEING SEEN          6155
ELOPED                           5710
OTHER                            4297
LEFT AGAINST MEDICAL ADVICE      1881
EXPIRED                           377
Name: count, dtype: int64

In [16]:
home_stays = ed_stays[ed_stays['disposition'] == 'HOME']
home_stays.shape

(241632, 9)

In [17]:
home_stays['subject_id'].nunique()

138273

In [18]:
# Filter ed_stays for 'ADMITTED' disposition
admitted_stays = ed_stays[ed_stays['disposition'] == 'ADMITTED']

# Count how many 'ADMITTED' stays have an associated hadm_id
total_admitted_stays = admitted_stays.shape[0]
admitted_with_hadm_id = admitted_stays['hadm_id'].notna().sum()

# Check how many of those hadm_ids in ed_stays are also in admissions
hadm_ids_in_ed_stays = admitted_stays['hadm_id'].dropna().unique()
hadm_ids_in_admissions = admissions['hadm_id'].unique()

# Count the number of hadm_ids from admitted stays that are also in admissions
hadm_ids_in_both = sum(hadm_id in hadm_ids_in_admissions for hadm_id in hadm_ids_in_ed_stays)

# Output the results
print(f"Total 'ADMITTED' stays: {total_admitted_stays}")
print(f"'ADMITTED' stays with hadm_id: {admitted_with_hadm_id}")
print(f"hadm_ids from admitted stays that are in admissions: {hadm_ids_in_both}")

Total 'ADMITTED' stays: 158010
'ADMITTED' stays with hadm_id: 157626
hadm_ids from admitted stays that are in admissions: 157601


In [19]:
(~admissions['edregtime'].isna()).sum()

379240

Will work with admitted patients, whose procedure IDs can be merged from the MIMIC hosp module to derive revascularization labels. This includes all ICU patients, so we don't need to handle that module seperately. Nearly all hospital admission ids from the admitted ED module patients appear in the hosp module. We will ignore the slight mismatch of 409 patients admitted according to the ED module but without hadm_id in the hosp module. We will use ED patients sent home and assign negative procedure labels as they have not undergone cardiac revascularization. 
Our screening populations consists of all 379240 patient presentations from the hosp module with an edregtime plus all 241632 sent home patient presentations from the ed module. Those are representative for all patients presenting to the emergency department whose medical precedures are given in the data.

In [20]:
lab = pd.read_csv(os.path.join(path_to_mimic, 'hosp/labevents.csv'))

In [21]:
lab.shape

(158478383, 16)

In [22]:
lab.columns

Index(['labevent_id', 'subject_id', 'hadm_id', 'specimen_id', 'itemid',
       'order_provider_id', 'charttime', 'storetime', 'value', 'valuenum',
       'valueuom', 'ref_range_lower', 'ref_range_upper', 'flag', 'priority',
       'comments'],
      dtype='object')

In [23]:
lab_ids = pd.read_csv(os.path.join(path_to_mimic, 'hosp/d_labitems.csv'))

In [24]:
lab_ids.shape

(1650, 4)

In [25]:
# Filter lab_ids for rows where the label contains 'troponin'
troponin_ids = lab_ids[lab_ids['label'].str.contains('troponin', case=False, na=False)]

# Extract the itemid values for these rows
troponin_itemids = troponin_ids['itemid'].unique()

In [26]:
# Filter the lab_ids DataFrame to get the rows for the specific itemids found
troponin_labels = lab_ids[lab_ids['itemid'].isin(troponin_itemids)]

# Extract the itemid and label columns to display the results
result = troponin_labels[['itemid', 'label']]

In [27]:
# Filter the lab_ids DataFrame to keep only rows where itemid is 51002
lab_troponin_t = lab[lab['itemid'] == 51002]
lab_troponin_t

,labevent_id,subject_id,hadm_id,specimen_id,itemid,order_provider_id,charttime,storetime,value,valuenum,valueuom,ref_range_lower,ref_range_upper,flag,priority,comments


In [28]:
# Filter the lab_ids DataFrame to keep only rows where itemid is 51003
lab_troponin_t = lab[lab['itemid'] == 51003]
lab_troponin_t.shape

(459872, 16)

In [29]:
# Filter rows where valuenum is NaN
nan_valuenum_rows = lab_troponin_t[lab_troponin_t['valuenum'].isna()]

# Get the value counts of the 'comments' column in these rows
comments_value_counts = nan_valuenum_rows['comments'].value_counts()

In [30]:
# Fill NaN values in valuenum where comments start with '<' or 'LESS' with 0.01
lab_troponin_t.loc[
    lab_troponin_t['valuenum'].isna() & lab_troponin_t['comments'].str.startswith(('<', 'LESS')),
    'valuenum'
] = 0.01

# Fill NaN values in valuenum where comments start with '>' or 'GREATER' with 25
lab_troponin_t.loc[
    lab_troponin_t['valuenum'].isna() & lab_troponin_t['comments'].str.startswith(('>', 'GREATER')),
    'valuenum'
] = 25

# Display the updated DataFrame or the first few rows to confirm
lab_troponin_t.shape

(459872, 16)

In [31]:
# Check how many NaN values are left in valuenum
nans_left = lab_troponin_t['valuenum'].isna().sum()
print(f"Number of NaN values left in valuenum: {nans_left}")

# Drop rows where valuenum is still NaN
lab_troponin_t_cleaned = lab_troponin_t.dropna(subset=['valuenum'])

# Check the shape of the cleaned DataFrame to confirm
print(f"Shape of DataFrame after dropping NaNs: {lab_troponin_t_cleaned.shape}")

Number of NaN values left in valuenum: 304
Shape of DataFrame after dropping NaNs: (459568, 16)


In [32]:
# Filter the lab_ids DataFrame to keep only rows where itemid is 51003
lab_troponin_i = lab[lab['itemid'] == 52642]
lab_troponin_i.shape

(670, 16)

In [33]:
# Filter lab_ids for rows where the label contains 'troponin'
ck_ids = lab_ids[lab_ids['label'].str.contains('Creatine Kinase', case=False, na=False)]

# Extract the itemid values for these rows
ck_itemids = ck_ids['itemid'].unique()

In [34]:
# Filter the lab_ids DataFrame to get the rows for the specific itemids found
ck_labels = lab_ids[lab_ids['itemid'].isin(ck_itemids)]

# Extract the itemid and label columns to display the results
result = ck_labels[['itemid', 'label']]

In [35]:
# Filter the lab_ids DataFrame to keep only rows where itemid is 51002
lab_ck = lab[lab['itemid'] == 50910]
lab_ck.shape

(334746, 16)

In [36]:
# Filter the lab_ids DataFrame to keep only rows where itemid is 51002
lab_ckmb = lab[lab['itemid'] == 50911]
lab_ckmb.shape

(266856, 16)

In [37]:
lab_ck_cleaned = lab_ck.dropna(subset=['valuenum'])

lab_ckmb_cleaned = lab_ckmb.dropna(subset=['valuenum'])

# Check the shape of the cleaned DataFrames to confirm
print(f"Shape of lab_ck after dropping NaNs: {lab_ck_cleaned.shape}")
print(f"Shape of lab_ckmb after dropping NaNs: {lab_ckmb_cleaned.shape}")

Shape of lab_ck after dropping NaNs: (334591, 16)
Shape of lab_ckmb after dropping NaNs: (218081, 16)


In [38]:
ecgs = pd.read_csv(os.path.join(path_to_mimic_ecg, 'record_list.csv'))

In [39]:
ecgs

,subject_id,study_id,file_name,ecg_time,path
0,10000032,40689238,40689238,2180-07-23 08:44:00,files/p1000/p10000032/s40689238/40689238
1,10000032,44458630,44458630,2180-07-23 09:54:00,files/p1000/p10000032/s44458630/44458630
2,10000032,49036311,49036311,2180-08-06 09:07:00,files/p1000/p10000032/s49036311/49036311
3,10000117,45090959,45090959,2181-03-04 17:14:00,files/p1000/p10000117/s45090959/45090959
4,10000117,48446569,48446569,2183-09-18 13:52:00,files/p1000/p10000117/s48446569/48446569
...,...,...,...,...,...
800030,19999840,48683947,48683947,2164-09-12 12:28:00,files/p1999/p19999840/s48683947/48683947
800031,19999840,41842293,41842293,2164-09-17 11:31:00,files/p1999/p19999840/s41842293/41842293
800032,19999987,41190887,41190887,2145-11-02 19:54:00,files/p1999/p19999987/s41190887/41190887
800033,19999987,45828463,45828463,2145-11-03 03:00:00,files/p1999/p19999987/s45828463/45828463


In [40]:
ecgs['ecg_time'] = pd.to_datetime(ecgs['ecg_time'])
admissions['edregtime'] = pd.to_datetime(admissions['edregtime'])

merged_df = pd.merge(ecgs, admissions[['subject_id', 'edregtime', 'hadm_id', 'race']], on='subject_id', how='left')

# Filter the ECGs to keep only those within 3 hours after ED presentation
admissions_ecgs = merged_df[
    (merged_df['ecg_time'] >= merged_df['edregtime']) &
    (merged_df['ecg_time'] <= merged_df['edregtime'] + pd.Timedelta(hours=3))
]

In [41]:
admissions_ecgs.shape

(152547, 8)

In [42]:
admissions_ecgs_sorted = admissions_ecgs.sort_values(by=['hadm_id', 'ecg_time'])

admissions_ecgs_earliest = admissions_ecgs_sorted.drop_duplicates(subset=['hadm_id'], keep='first')

In [43]:
admissions_ecgs_earliest.shape

(137331, 8)

In [44]:
ed_stays['intime'] = pd.to_datetime(ed_stays['intime'])

merged_df = pd.merge(ecgs, ed_stays[['subject_id', 'intime', 'stay_id', 'disposition', 'arrival_transport', 'race']], on='subject_id', how='left')

# Filter the ECGs to keep only those within 3 hours after ED presentation
ed_ecgs = merged_df[
    (merged_df['ecg_time'] >= merged_df['intime']) &
    (merged_df['ecg_time'] <= merged_df['intime'] + pd.Timedelta(hours=3))
]

# Filter the ed_ecgs DataFrame to keep only rows where disposition is 'HOME'
ed_ecgs = ed_ecgs[ed_ecgs['disposition'] == 'HOME']
ed_ecgs = ed_ecgs.drop(columns=['disposition'])
ed_ecgs = ed_ecgs.rename(columns={'intime': 'edregtime', 'stay_id':'ed_stay_id'})

In [45]:
ed_ecgs.shape

(61350, 9)

In [46]:
ed_ecgs_sorted = ed_ecgs.sort_values(by=['ed_stay_id', 'ecg_time'])

ed_ecgs_earliest = ed_ecgs_sorted.drop_duplicates(subset=['ed_stay_id'], keep='first')

In [47]:
ed_ecgs_earliest.shape

(57821, 9)

In [48]:
common_study_ids = ed_ecgs_earliest['study_id'].isin(admissions_ecgs_earliest['study_id'])
common_count = common_study_ids.sum()

print(f"Number of common study_ids: {common_count}")

Number of common study_ids: 13745


In [49]:
combined_ecgs = pd.concat([ed_ecgs_earliest, admissions_ecgs_earliest], ignore_index=True)

In [50]:
combined_ecgs.shape

(195152, 10)

In [51]:
# Sort combined_ecgs by study_id and other relevant columns
combined_ecgs_sorted = combined_ecgs.sort_values(
    by=['study_id', 'ecg_time', 'ed_stay_id', 'hadm_id'],
    ascending=[True, True, False, False]
)

combined_ecgs_filled = combined_ecgs_sorted.groupby('study_id', as_index=False).apply(lambda x: x.ffill().bfill())
combined_ecgs_cleaned = combined_ecgs_filled.drop_duplicates(subset=['study_id'], keep='first')

combined_ecgs_cleaned.shape

/tmp/ipykernel_3335723/2337414888.py:7: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  combined_ecgs_filled = combined_ecgs_sorted.groupby('study_id', as_index=False).apply(lambda x: x.ffill().bfill())
/tmp/ipykernel_3335723/2337414888.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  combined_ecgs_filled = combined_ecgs_sorted.groupby('study_id', as_index=False).apply(lambda x: x.ffill().bfill())


(180686, 10)

In [52]:
combined_ecgs_cleaned['subject_id'].nunique()

97982

In [53]:
procedures = pd.read_csv(os.path.join(path_to_mimic, 'hosp/procedures_icd.csv'))
procedures.shape

(859655, 6)

In [54]:
revasc_icd_codes = pd.read_excel(os.path.join(path_to_local_files, 'revasc_d_icd_procedures.xlsx'))
revasc_icd_codes

,icd_code,icd_version,long_title
0,66,9,Percutaneous transluminal coronary angioplasty...
1,210083,10,"Bypass Coronary Artery, One Artery from Corona..."
2,210088,10,"Bypass Coronary Artery, One Artery from Right ..."
3,210089,10,"Bypass Coronary Artery, One Artery from Left I..."
4,021008C,10,"Bypass Coronary Artery, One Artery from Thorac..."
...,...,...,...
578,3E07316,10,Introduction of Recombinant Human-activated Pr...
579,3E07317,10,Introduction of Other Thrombolytic into Corona...
580,3E073GC,10,Introduction of Other Therapeutic Substance in...
581,3E073KZ,10,Introduction of Other Diagnostic Substance int...


In [55]:
cabg_icd_codes = pd.read_excel(os.path.join(path_to_local_files, 'cabg_d_icd_procedures.xlsx'))
cabg_icd_codes

,icd_code,icd_version,long_title
0,210083,10,"Bypass Coronary Artery, One Artery from Corona..."
1,210088,10,"Bypass Coronary Artery, One Artery from Right ..."
2,210089,10,"Bypass Coronary Artery, One Artery from Left I..."
3,021008C,10,"Bypass Coronary Artery, One Artery from Thorac..."
4,021008F,10,"Bypass Coronary Artery, One Artery from Abdomi..."
...,...,...,...
394,3639,9,Other heart revascularization
395,3E07017,10,Introduction of Other Thrombolytic into Corona...
396,3E070GC,10,Introduction of Other Therapeutic Substance in...
397,3E070KZ,10,Introduction of Other Diagnostic Substance int...


In [56]:
pci_icd_codes = pd.read_excel(os.path.join(path_to_local_files, 'pci_d_icd_procedures.xlsx'))
pci_icd_codes

,icd_code,icd_version,long_title
0,66,9,Percutaneous transluminal coronary angioplasty...
1,270346,10,"Dilation of Coronary Artery, One Artery, Bifur..."
2,027034Z,10,"Dilation of Coronary Artery, One Artery with D..."
3,270356,10,"Dilation of Coronary Artery, One Artery, Bifur..."
4,027035Z,10,"Dilation of Coronary Artery, One Artery with T..."
...,...,...,...
179,3E07316,10,Introduction of Recombinant Human-activated Pr...
180,3E07317,10,Introduction of Other Thrombolytic into Corona...
181,3E073GC,10,Introduction of Other Therapeutic Substance in...
182,3E073KZ,10,Introduction of Other Diagnostic Substance int...


In [57]:
cag_icd_codes = pd.read_excel(os.path.join(path_to_local_files, 'cag_d_icd_procedures.xlsx'))
cag_icd_codes

,icd_code,icd_version,long_title
0,24,9,Intravascular imaging of coronary vessels
1,59,9,Intravascular pressure measurement of coronary...
2,3824,9,Intravascular imaging of coronary vessel(s) by...
3,3825,9,Intravascular imaging of non-coronary vessel(s...
4,8852,9,Angiocardiography of right heart structures
...,...,...,...
79,B2181ZZ,10,Fluoroscopy of Left Internal Mammary Bypass Gr...
80,B218YZZ,10,Fluoroscopy of Left Internal Mammary Bypass Gr...
81,B21F0ZZ,10,Fluoroscopy of Other Bypass Graft using High O...
82,B21F1ZZ,10,Fluoroscopy of Other Bypass Graft using Low Os...


In [58]:
procedures_grouped = procedures.groupby('hadm_id')['icd_code'].apply(list).reset_index()
combined_ecgs_cleaned_with_procedures = pd.merge(combined_ecgs_cleaned, procedures_grouped, on='hadm_id', how='left')

combined_ecgs_cleaned_with_procedures['icd_code'] = combined_ecgs_cleaned_with_procedures['icd_code'].apply(lambda x: x if isinstance(x, list) else [])

In [59]:
procedures_grouped.shape

(287504, 2)

In [60]:
revasc_code_set = set(revasc_icd_codes['icd_code'])

def check_revasc(icd_list):
    return 1 if any(code in revasc_code_set for code in icd_list) else 0

# Apply the function to each row's 'icd_code' column to create the 'PCI_OR_CABG' column
combined_ecgs_cleaned_with_procedures['REVASC'] = combined_ecgs_cleaned_with_procedures['icd_code'].apply(check_revasc)


In [61]:
combined_ecgs_cleaned_with_procedures['REVASC'].value_counts()

REVASC
0    179602
1      1084
Name: count, dtype: int64

In [62]:
cabg_code_set = set(cabg_icd_codes['icd_code'])

# Define a function to check if any of the ICD codes in the list are in the PCI/CABG set
def check_cabg(icd_list):
    return 1 if any(code in cabg_code_set for code in icd_list) else 0

# Apply the function to each row's 'icd_code' column to create the 'PCI_OR_CABG' column
combined_ecgs_cleaned_with_procedures['CABG'] = combined_ecgs_cleaned_with_procedures['icd_code'].apply(check_cabg)


In [63]:
combined_ecgs_cleaned_with_procedures['CABG'].value_counts()

CABG
0    180433
1       253
Name: count, dtype: int64

In [64]:
pci_code_set = set(pci_icd_codes['icd_code'])

def check_pci(icd_list):
    return 1 if any(code in pci_code_set for code in icd_list) else 0

# Apply the function to each row's 'icd_code' column to create the 'PCI_OR_CABG' column
combined_ecgs_cleaned_with_procedures['PCI'] = combined_ecgs_cleaned_with_procedures['icd_code'].apply(check_pci)

In [65]:
combined_ecgs_cleaned_with_procedures['PCI'].value_counts()

PCI
0    179851
1       835
Name: count, dtype: int64

In [66]:
combined_ecgs_cleaned_with_procedures[(combined_ecgs_cleaned_with_procedures['PCI']==1) & (combined_ecgs_cleaned_with_procedures['CABG']==1)].shape

(4, 14)

In [67]:
cag_code_set = set(cag_icd_codes['icd_code'])

def check_cag(icd_list):
    return 1 if any(code in cag_code_set for code in icd_list) else 0

# Apply the function to each row's 'icd_code' column to create the 'PCI_OR_CABG' column
combined_ecgs_cleaned_with_procedures['CAG'] = combined_ecgs_cleaned_with_procedures['icd_code'].apply(check_cag)

In [68]:
combined_ecgs_cleaned_with_procedures['CAG'].value_counts()

CAG
0    178583
1      2103
Name: count, dtype: int64

In [69]:
# Drop the 'icd_code' column
ecg_dataset = combined_ecgs_cleaned_with_procedures.drop(columns=['icd_code'])
ecg_dataset.shape

(180686, 14)

In [70]:
patients = pd.read_csv(os.path.join(path_to_mimic, 'hosp/patients.csv'))
patients.shape

(364627, 6)

In [71]:
ecg_dataset = ecg_dataset.merge(patients[['subject_id', 'gender', 'anchor_age']], on='subject_id', how='left')
ecg_dataset.shape

(180686, 16)

In [72]:
merged_df = pd.merge(ecg_dataset, lab_troponin_t_cleaned, on='subject_id', suffixes=('_ecg', '_troponin'))

In [73]:
merged_df.shape

(1799240, 31)

In [74]:
filtered_df = merged_df[
    (merged_df['charttime'] >= merged_df['edregtime']) &
    (merged_df['charttime'] <= merged_df['edregtime'] + pd.Timedelta(hours=3))
]

In [75]:
filtered_df.shape

(86080, 31)

In [76]:
filtered_df.columns

Index(['subject_id', 'study_id', 'file_name', 'ecg_time', 'path', 'edregtime',
       'ed_stay_id', 'arrival_transport', 'race', 'hadm_id_ecg', 'REVASC',
       'CABG', 'PCI', 'CAG', 'gender', 'anchor_age', 'labevent_id',
       'hadm_id_troponin', 'specimen_id', 'itemid', 'order_provider_id',
       'charttime', 'storetime', 'value', 'valuenum', 'valueuom',
       'ref_range_lower', 'ref_range_upper', 'flag', 'priority', 'comments'],
      dtype='object')

In [77]:
filtered_df = filtered_df.sort_values(by=['study_id', 'charttime'])

grouped = filtered_df.groupby('study_id')['valuenum'].apply(lambda x: list(x[:3])).reset_index()
grouped.shape

(84881, 2)

In [78]:
# Create columns for the first, second, and third troponin values
grouped['first_troponin_t'] = grouped['valuenum'].apply(lambda x: x[0] if len(x) > 0 else np.nan)
grouped['second_troponin_t'] = grouped['valuenum'].apply(lambda x: x[1] if len(x) > 1 else np.nan)
grouped['third_troponin_t'] = grouped['valuenum'].apply(lambda x: x[2] if len(x) > 2 else np.nan)

# Calculate troponin_t_delta (difference between the highest and lowest values)
grouped['troponin_t_delta'] = grouped['valuenum'].apply(lambda x: max(x) - min(x) if len(x) > 1 else np.nan)
grouped.shape

(84881, 6)

In [79]:
ecg_dataset = pd.merge(ecg_dataset, grouped[['study_id', 'first_troponin_t', 'second_troponin_t', 'third_troponin_t', 'troponin_t_delta']], on='study_id', how='left')

ecg_dataset.iloc[:, -4:].shape

(180686, 4)

In [80]:
ecg_dataset['first_troponin_t'].isna().sum()

95805

In [81]:
merged_df = pd.merge(ecg_dataset, lab_ck_cleaned, on='subject_id', suffixes=('_ecg', '_ck'))

filtered_df = merged_df[
    (merged_df['charttime'] >= merged_df['edregtime']) &
    (merged_df['charttime'] <= merged_df['edregtime'] + pd.Timedelta(hours=3))
]

filtered_df = filtered_df.sort_values(by=['study_id', 'charttime'])

grouped = filtered_df.groupby('study_id')['valuenum'].apply(lambda x: list(x[:3])).reset_index()

# Create columns for the first, second, and third troponin values
grouped['first_ck'] = grouped['valuenum'].apply(lambda x: x[0] if len(x) > 0 else np.nan)
grouped['second_ck'] = grouped['valuenum'].apply(lambda x: x[1] if len(x) > 1 else np.nan)
grouped['third_ck'] = grouped['valuenum'].apply(lambda x: x[2] if len(x) > 2 else np.nan)

# Calculate troponin_t_delta (difference between the highest and lowest values)
grouped['ck_delta'] = grouped['valuenum'].apply(lambda x: max(x) - min(x) if len(x) > 1 else np.nan)

# Merge these new columns back into ecg_dataset
ecg_dataset = pd.merge(ecg_dataset, grouped[['study_id', 'first_ck', 'second_ck', 'third_ck', 'ck_delta']], on='study_id', how='left')

In [82]:
ecg_dataset['first_ck'].isna().sum()

147454

In [83]:
# Merge the DataFrames on subject_id
merged_df = pd.merge(ecg_dataset, lab_ckmb_cleaned, on='subject_id', suffixes=('_ecg', '_ckmb'))

filtered_df = merged_df[
    (merged_df['charttime'] >= merged_df['edregtime']) &
    (merged_df['charttime'] <= merged_df['edregtime'] + pd.Timedelta(hours=3))
]

# SSort the DataFrame by study_id and charttime
filtered_df = filtered_df.sort_values(by=['study_id', 'charttime'])

# roup by study_id and get the first, second, and third troponin values
grouped = filtered_df.groupby('study_id')['valuenum'].apply(lambda x: list(x[:3])).reset_index()

grouped['first_ckmb'] = grouped['valuenum'].apply(lambda x: x[0] if len(x) > 0 else np.nan)
grouped['second_ckmb'] = grouped['valuenum'].apply(lambda x: x[1] if len(x) > 1 else np.nan)
grouped['third_ckmb'] = grouped['valuenum'].apply(lambda x: x[2] if len(x) > 2 else np.nan)

# Calculate troponin_t_delta (difference between the highest and lowest values)
grouped['ckmb_delta'] = grouped['valuenum'].apply(lambda x: max(x) - min(x) if len(x) > 1 else np.nan)

ecg_dataset = pd.merge(ecg_dataset, grouped[['study_id', 'first_ckmb', 'second_ckmb', 'third_ckmb', 'ckmb_delta']], on='study_id', how='left')

In [84]:
machine_measurements = pd.read_csv(os.path.join(path_to_mimic_ecg, 'machine_measurements.csv'))
machine_measurements.shape

/tmp/ipykernel_3335723/1390149727.py:1: DtypeWarning: Columns (16,17,18,19,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  machine_measurements = pd.read_csv(os.path.join(path_to_mimic_ecg, 'machine_measurements.csv'))


(800035, 33)

In [85]:
machine_measurements['rr_interval_seconds'] = machine_measurements['rr_interval'] / 1000
machine_measurements['ventricular_rate'] = 60 / machine_measurements['rr_interval_seconds']

machine_measurements['pr_interval'] = machine_measurements['qrs_onset'] - machine_measurements['p_onset']
machine_measurements['p_wave_duration'] = machine_measurements['p_end'] - machine_measurements['p_onset']
machine_measurements['qrs_duration'] = machine_measurements['qrs_end'] - machine_measurements['qrs_onset']
machine_measurements['qt_interval'] = machine_measurements['t_end'] - machine_measurements['qrs_onset']
machine_measurements.columns

Index(['subject_id', 'study_id', 'cart_id', 'ecg_time', 'report_0', 'report_1',
       'report_2', 'report_3', 'report_4', 'report_5', 'report_6', 'report_7',
       'report_8', 'report_9', 'report_10', 'report_11', 'report_12',
       'report_13', 'report_14', 'report_15', 'report_16', 'report_17',
       'bandwidth', 'filtering', 'rr_interval', 'p_onset', 'p_end',
       'qrs_onset', 'qrs_end', 't_end', 'p_axis', 'qrs_axis', 't_axis',
       'rr_interval_seconds', 'ventricular_rate', 'pr_interval',
       'p_wave_duration', 'qrs_duration', 'qt_interval'],
      dtype='object')

In [86]:
machine_measurements['pr_interval'].describe()

count    800035.000000
mean      -4419.455578
std       10790.427735
min      -29999.000000
25%         128.000000
50%         153.000000
75%         176.000000
max       29863.000000
Name: pr_interval, dtype: float64

In [87]:
# Checks for plausibility of values
machine_measurements['pr_interval'] = machine_measurements['pr_interval'].apply(lambda x: np.nan if x < 20 or x > 500 else x)
machine_measurements['p_wave_duration'] = machine_measurements['p_wave_duration'].apply(lambda x: np.nan if x <= 20 or x > 300 else x)
machine_measurements['qrs_duration'] = machine_measurements['qrs_duration'].apply(lambda x: np.nan if x < 50 or x > 300 else x)
machine_measurements['qt_interval'] = machine_measurements['qt_interval'].apply(lambda x: np.nan if x < 200 or x > 700 else x)

In [88]:
# Calculate qtc_interval using Bazett's formula
machine_measurements['qtc_interval'] = machine_measurements['qt_interval'] / np.sqrt(60 / machine_measurements['ventricular_rate'])

# Drop the temporary rr_interval_sec column if not needed
machine_measurements.drop(columns=['rr_interval_seconds'], inplace=True)

In [89]:
# Replace p_axis with NaN if p_wave_duration is NaN
machine_measurements['p_axis'] = machine_measurements.apply(
    lambda row: np.nan if pd.isna(row['p_wave_duration']) else row['p_axis'], axis=1
)

# Replace qrs_axis with NaN if qrs_duration is NaN
machine_measurements['qrs_axis'] = machine_measurements.apply(
    lambda row: np.nan if pd.isna(row['qrs_duration']) else row['qrs_axis'], axis=1
)

# Replace t_axis with NaN if qt_interval is NaN
machine_measurements['t_axis'] = machine_measurements.apply(
    lambda row: np.nan if pd.isna(row['qt_interval']) else row['t_axis'], axis=1
)

In [90]:
machine_measurements['qrs_axis'].describe()

count    798267.000000
mean         58.523329
std        1149.475732
min        -180.000000
25%         -15.000000
50%          13.000000
75%          45.000000
max       29999.000000
Name: qrs_axis, dtype: float64

In [91]:
machine_measurements['p_axis'] = machine_measurements['p_axis'].apply(lambda x: np.nan if x < -90 or x > 270 else x)
machine_measurements['qrs_axis'] = machine_measurements['qrs_axis'].apply(lambda x: np.nan if x < -90 or x > 270 else x)
machine_measurements['t_axis'] = machine_measurements['t_axis'].apply(lambda x: np.nan if x < -90 or x > 270 else x)

In [92]:
machine_measurements.shape

(800035, 39)

In [93]:
columns_to_merge = [
    'study_id', 'report_0', 'ventricular_rate', 'pr_interval', 
    'p_wave_duration', 'qrs_duration', 'qt_interval', 'qtc_interval',
    'p_axis', 'qrs_axis', 't_axis'
]

machine_measurements_to_merge = machine_measurements[columns_to_merge]

ecg_dataset = pd.merge(ecg_dataset, machine_measurements_to_merge, on='study_id', how='left')
ecg_dataset.shape

(180686, 38)

In [94]:
ecg_dataset = ecg_dataset.rename(columns={'report_0': 'rhythm'})

ecg_dataset['sinus_rhythm'] = ecg_dataset['rhythm'].apply(lambda x: 1 if pd.notna(x) and 'sinus rhythm' in x.lower() else 0)
ecg_dataset['sinus_rhythm'].value_counts()

sinus_rhythm
1    107609
0     73077
Name: count, dtype: int64

In [95]:
diagnoses_hosp = pd.read_csv(os.path.join(path_to_mimic, 'hosp/diagnoses_icd.csv'))
diagnoses_hosp.shape

(6364520, 5)

In [96]:
diagnoses_hosp_ids = pd.read_csv(os.path.join(path_to_mimic, 'hosp/d_icd_diagnoses.csv'))
diagnoses_hosp_ids.shape

(112107, 3)

In [97]:
diagnoses_ed = pd.read_csv(os.path.join(path_to_mimic_ed, 'ed/diagnosis.csv'))
diagnoses_ed.shape

(899050, 6)

In [98]:
diabetes_ids = diagnoses_hosp_ids[diagnoses_hosp_ids['long_title'].str.contains('diabetes', case=False, na=False)]

diabetes_icd_codes = diabetes_ids['icd_code'].unique()

In [99]:
diabetes_labels = diagnoses_hosp_ids[diagnoses_hosp_ids['icd_code'].isin(diabetes_icd_codes)]

result = diabetes_labels[['icd_code', 'long_title']]

We will export this list for manual filtering of relevant ICD codes for diabetes. We have the same approach for all relevant diagnoses in the following

In [100]:
output_path = os.path.join(path_to_local_files, 'diabetes_icd_codes.xlsx')
result.to_excel(output_path, index=False)

In [101]:
hypertension_ids = diagnoses_hosp_ids[diagnoses_hosp_ids['long_title'].str.contains('hypertension', case=False, na=False)]

hypertension_icd_codes = hypertension_ids['icd_code'].unique()

print("ICD Codes with 'hypertension' in long_title:", hypertension_icd_codes)

hypertension_labels = diagnoses_hosp_ids[diagnoses_hosp_ids['icd_code'].isin(hypertension_icd_codes)]

result = hypertension_labels[['icd_code', 'long_title']]

ICD Codes with 'hypertension' in long_title: ['3482' '36504' '4010' '4011' '4019' '40501' '40509' '40511' '40519'
 '40591' '40599' '4160' '45930' '45931' '45932' '45933' '45939' '5723'
 '64200' '64201' '64202' '64203' '64204' '64210' '64211' '64212' '64213'
 '64214' '64220' '64221' '64222' '64223' '64224' '64230' '64231' '64232'
 '64233' '64234' '64270' '64271' '64272' '64273' '64274' '64290' '64291'
 '64292' '64293' '64294' '64610' '64611' '64612' '64613' '64614' '64620'
 '64621' '64622' '64623' '64624' '7962' '99791' 'G932' 'H4005' 'H40051'
 'H40052' 'H40053' 'H40059' 'I10' 'I15' 'I150' 'I151' 'I152' 'I158' 'I159'
 'I270' 'I272' 'I2720' 'I2721' 'I2722' 'I2723' 'I2724' 'I2729' 'I873'
 'I8730' 'I87301' 'I87302' 'I87303' 'I87309' 'I8731' 'I87311' 'I87312'
 'I87313' 'I87319' 'I8732' 'I87321' 'I87322' 'I87323' 'I87329' 'I8733'
 'I87331' 'I87332' 'I87333' 'I87339' 'I8739' 'I87391' 'I87392' 'I87393'
 'I87399' 'I973' 'K766' 'O10' 'O100' 'O1001' 'O10011' 'O10012' 'O10013'
 'O10019' 'O1002' 'O

In [102]:
output_path = os.path.join(path_to_local_files, 'hypertension_icd_codes.xlsx')
result.to_excel(output_path, index=False)

In [103]:
hyperlipidemia_ids = diagnoses_hosp_ids[
    diagnoses_hosp_ids['long_title'].str.contains('hyperlipidemia|hypercholesterolemia|hyperglyceridemia|cholesterol|Lipoprotein', case=False, na=False)
]

hyperlipidemia_icd_codes = hyperlipidemia_ids['icd_code'].unique()


# Filter the diagnoses_hosp_ids DataFrame to get the rows for the specific itemids found
hyperlipidemia_labels = diagnoses_hosp_ids[diagnoses_hosp_ids['icd_code'].isin(hyperlipidemia_icd_codes)]

result = hyperlipidemia_labels[['icd_code', 'long_title']]

In [104]:
output_path = os.path.join(path_to_local_files, 'hyperlipidemia_icd_codes.xlsx')
result.to_excel(output_path, index=False)

In [105]:
old_mi_ids = diagnoses_hosp_ids[diagnoses_hosp_ids['long_title'].str.contains('old myocardial infarction', case=False, na=False)]

# Extract the unique icd_code values for these rows
old_mi_icd_codes = old_mi_ids['icd_code'].unique()

print("ICD Codes with 'old myocardial infarction' in long_title:", old_mi_icd_codes)

# Filter the diagnoses_hosp_ids DataFrame to get the rows for the specific icd_codes found
old_mi_labels = diagnoses_hosp_ids[diagnoses_hosp_ids['icd_code'].isin(old_mi_icd_codes)]

result = old_mi_labels[['icd_code', 'long_title']]

output_path = os.path.join(path_to_local_files, 'old_mi_icd_codes.xlsx')
result.to_excel(output_path, index=False)

ICD Codes with 'old myocardial infarction' in long_title: ['412' 'I252']


In [106]:
relevant_diabetes_icd_codes = pd.read_excel(os.path.join(path_to_local_files, 'relevant_diabetes_icd_codes.xlsx'))
relevant_diabetes_icd_codes.shape

(695, 2)

In [107]:
relevant_hypertension_icd_codes = pd.read_excel(os.path.join(path_to_local_files, 'relevant_hypertension_icd_codes.xlsx'))
relevant_hyperlipidemia_icd_codes = pd.read_excel(os.path.join(path_to_local_files, 'relevant_hyperlipidemia_icd_codes.xlsx'))
relevant_old_mi_icd_codes = pd.read_excel(os.path.join(path_to_local_files, 'relevant_old_mi_icd_codes.xlsx'))

In [108]:
# Get the relevant ICD codes
diabetes_icd_codes = relevant_diabetes_icd_codes['icd_code'].unique()
hypertension_icd_codes = relevant_hypertension_icd_codes['icd_code'].unique()
hyperlipidemia_icd_codes = relevant_hyperlipidemia_icd_codes['icd_code'].unique()
old_mi_icd_codes = relevant_old_mi_icd_codes['icd_code'].unique()

# Create a flag in diagnoses_hosp indicating if the icd_code is related to disease of interest
diagnoses_hosp['diabetes_flag'] = diagnoses_hosp['icd_code'].isin(diabetes_icd_codes).astype(int)
diagnoses_hosp['hypertension_flag'] = diagnoses_hosp['icd_code'].isin(hypertension_icd_codes).astype(int)
diagnoses_hosp['hyperlipidemia_flag'] = diagnoses_hosp['icd_code'].isin(hyperlipidemia_icd_codes).astype(int)
diagnoses_hosp['old_mi_flag'] = diagnoses_hosp['icd_code'].isin(old_mi_icd_codes).astype(int)

# Aggregate by hadm_id to determine if any diagnosis linked to the hadm_id is related to disease of interest
hadm_diabetes = diagnoses_hosp.groupby('hadm_id')['diabetes_flag'].max().reset_index()
hadm_hypertension = diagnoses_hosp.groupby('hadm_id')['hypertension_flag'].max().reset_index()
hadm_hyperlipidemia = diagnoses_hosp.groupby('hadm_id')['hyperlipidemia_flag'].max().reset_index()
hadm_old_mi = diagnoses_hosp.groupby('hadm_id')['old_mi_flag'].max().reset_index()

# Merge this result with the ecg_dataset on hadm_id
ecg_dataset = pd.merge(ecg_dataset, hadm_diabetes, on='hadm_id', how='left')
ecg_dataset = pd.merge(ecg_dataset, hadm_hypertension, on='hadm_id', how='left')
ecg_dataset = pd.merge(ecg_dataset, hadm_hyperlipidemia, on='hadm_id', how='left')
ecg_dataset = pd.merge(ecg_dataset, hadm_old_mi, on='hadm_id', how='left')

# Identify the NaN values in hypertension, chronic_ihd, and diabetes columns
nan_hypertension = ecg_dataset[ecg_dataset['hypertension_flag'].isna()]
nan_hyperlipidemia = ecg_dataset[ecg_dataset['hyperlipidemia_flag'].isna()]
nan_old_mi = ecg_dataset[ecg_dataset['old_mi_flag'].isna()]
nan_diabetes = ecg_dataset[ecg_dataset['diabetes_flag'].isna()]

# Merge the ecg_dataset with diagnoses_ed based on stay_id
hypertension_ed = pd.merge(nan_hypertension, diagnoses_ed, left_on='ed_stay_id', right_on='stay_id', how='left')
hyperlipidemia_ed = pd.merge(nan_hyperlipidemia, diagnoses_ed, left_on='ed_stay_id', right_on='stay_id', how='left')
old_mi_ed = pd.merge(nan_old_mi, diagnoses_ed, left_on='ed_stay_id', right_on='stay_id', how='left')
diabetes_ed = pd.merge(nan_diabetes, diagnoses_ed, left_on='ed_stay_id', right_on='stay_id', how='left')

# Create flags based on diagnoses in diagnoses_ed
hypertension_ed['hypertension_flag_ed'] = hypertension_ed['icd_code'].isin(hypertension_icd_codes).astype(int)
hyperlipidemia_ed['hyperlipidemia_flag_ed'] = hyperlipidemia_ed['icd_code'].isin(hyperlipidemia_icd_codes).astype(int)
old_mi_ed['old_mi_flag_ed'] = old_mi_ed['icd_code'].isin(old_mi_icd_codes).astype(int)
diabetes_ed['diabetes_flag_ed'] = diabetes_ed['icd_code'].isin(diabetes_icd_codes).astype(int)

# Aggregate to see if there is any relevant diagnosis per stay_id
hypertension_fill = hypertension_ed.groupby('ed_stay_id')['hypertension_flag_ed'].max().reset_index()
hyperlipidemia_fill = hyperlipidemia_ed.groupby('ed_stay_id')['hyperlipidemia_flag_ed'].max().reset_index()
old_mi_fill = old_mi_ed.groupby('ed_stay_id')['old_mi_flag_ed'].max().reset_index()
diabetes_fill = diabetes_ed.groupby('ed_stay_id')['diabetes_flag_ed'].max().reset_index()

# Merge these back into ecg_dataset to fill in the NaNs
ecg_dataset = pd.merge(ecg_dataset, hypertension_fill, on='ed_stay_id', how='left')
ecg_dataset = pd.merge(ecg_dataset, hyperlipidemia_fill, on='ed_stay_id', how='left')
ecg_dataset = pd.merge(ecg_dataset, old_mi_fill, on='ed_stay_id', how='left')
ecg_dataset = pd.merge(ecg_dataset, diabetes_fill, on='ed_stay_id', how='left')

# Fill the NaNs in the original columns with values from the diagnoses_ed lookup
ecg_dataset['hypertension_flag'] = ecg_dataset['hypertension_flag'].fillna(ecg_dataset['hypertension_flag_ed'])
ecg_dataset['hyperlipidemia_flag'] = ecg_dataset['hyperlipidemia_flag'].fillna(ecg_dataset['hyperlipidemia_flag_ed'])
ecg_dataset['old_mi_flag'] = ecg_dataset['old_mi_flag'].fillna(ecg_dataset['old_mi_flag_ed'])
ecg_dataset['diabetes_flag'] = ecg_dataset['diabetes_flag'].fillna(ecg_dataset['diabetes_flag_ed'])

# replace remaining NaNs with 0 (in case there were no relevant diagnoses anywhere)
ecg_dataset['hypertension_flag'] = ecg_dataset['hypertension_flag'].fillna(0).astype(int)
ecg_dataset['hyperlipidemia_flag'] = ecg_dataset['hyperlipidemia_flag'].fillna(0).astype(int)
ecg_dataset['old_mi_flag'] = ecg_dataset['old_mi_flag'].fillna(0).astype(int)
ecg_dataset['diabetes_flag'] = ecg_dataset['diabetes_flag'].fillna(0).astype(int)

# Rename the flag columns to their final names
ecg_dataset = ecg_dataset.rename(columns={
    'diabetes_flag': 'diabetes',
    'hypertension_flag': 'hypertension',
    'hyperlipidemia_flag': 'hyperlipidemia',
    'old_mi_flag': 'old_mi'
})

# Drop the temporary _ed columns
ecg_dataset.drop(columns=['hypertension_flag_ed', 'hyperlipidemia_flag_ed', 
                          'old_mi_flag_ed', 'diabetes_flag_ed'], inplace=True)

In [109]:
ecg_dataset['hyperlipidemia'].value_counts()

hyperlipidemia
0    124573
1     56113
Name: count, dtype: int64

In [110]:
ecg_dataset['old_mi'].value_counts()

old_mi
0    177053
1      3633
Name: count, dtype: int64

In [111]:
ecg_dataset.columns

Index(['subject_id', 'study_id', 'file_name', 'ecg_time', 'path', 'edregtime',
       'ed_stay_id', 'arrival_transport', 'race', 'hadm_id', 'REVASC', 'CABG',
       'PCI', 'CAG', 'gender', 'anchor_age', 'first_troponin_t',
       'second_troponin_t', 'third_troponin_t', 'troponin_t_delta', 'first_ck',
       'second_ck', 'third_ck', 'ck_delta', 'first_ckmb', 'second_ckmb',
       'third_ckmb', 'ckmb_delta', 'rhythm', 'ventricular_rate', 'pr_interval',
       'p_wave_duration', 'qrs_duration', 'qt_interval', 'qtc_interval',
       'p_axis', 'qrs_axis', 't_axis', 'sinus_rhythm', 'diabetes',
       'hypertension', 'hyperlipidemia', 'old_mi'],
      dtype='object')

In [112]:
file_path = os.path.join(path_to_local_files, 'mimic_ecg_dataset.csv')

ecg_dataset.to_csv(file_path, index=False)

In [113]:
ecg_dataset = pd.read_csv(os.path.join(path_to_local_files, 'mimic_ecg_dataset.csv'))

In [114]:
triage = pd.read_csv(os.path.join(path_to_mimic_ed, 'ed/triage.csv'))
triage.shape

(425087, 11)

In [115]:
# Set the display option to show all rows
#pd.set_option('display.max_rows', None)

# Print the full value counts
#print(triage['chiefcomplaint'].value_counts())

# Reset the display option to the default
pd.reset_option('display.max_rows')

In [116]:
triage['chiefcomplaint'].value_counts()

chiefcomplaint
Chest pain                     11301
Abd pain                       10771
Dyspnea                         6003
s/p Fall                        4965
ABD PAIN                        4957
                               ...  
Hyponatremia, Hyperglycemia        1
Chest pain, Chills, Dyspnea        1
SBO-TRANSFER                       1
DYSURIA, FEVER                     1
? SEIZURE TODAY                    1
Name: count, Length: 60406, dtype: int64

In [117]:
ecg_dataset.shape

(180686, 43)

In [118]:
# Filter the rows where hadm_id is not null but ed_stay_id is null
rows_with_hadm_no_ed_stay = ecg_dataset[(ecg_dataset['hadm_id'].notnull()) & (ecg_dataset['ed_stay_id'].isnull())]

count_rows_with_hadm_no_ed_stay = rows_with_hadm_no_ed_stay.shape[0]
print(f"Number of rows with a value in hadm_id but not in ed_stay_id: {count_rows_with_hadm_no_ed_stay}")

Number of rows with a value in hadm_id but not in ed_stay_id: 122868


In [119]:
rows_with_ed_stay_no_hadm = ecg_dataset[(ecg_dataset['ed_stay_id'].notnull()) & (ecg_dataset['hadm_id'].isnull())]

count_rows_with_ed_stay_no_hadm = rows_with_ed_stay_no_hadm.shape[0]
print(f"Number of rows with a value in ed_stay_id but not in hadm_id: {count_rows_with_ed_stay_no_hadm}")

Number of rows with a value in ed_stay_id but not in hadm_id: 44073


In [120]:
ecg_dataset['ed_stay_id'].isnull().sum()

122868

In [121]:
ecg_dataset['hadm_id'].isnull().sum()

44073

In [122]:
rows_with_both_ids = ecg_dataset[(ecg_dataset['hadm_id'].notnull()) & (ecg_dataset['ed_stay_id'].notnull())]

count_rows_with_both_ids = rows_with_both_ids.shape[0]
print(f"Number of rows with values in both hadm_id and ed_stay_id: {count_rows_with_both_ids}")

Number of rows with values in both hadm_id and ed_stay_id: 13745


In [123]:
#  Sort ed_stays by 'hadm_id' and 'intime' to ensure the earliest 'intime' is kept
ed_stays_sorted = ed_stays.sort_values(by=['hadm_id', 'intime'])

# Drop duplicates based on 'hadm_id', keeping the first occurrence (which is now the earliest intime)
ed_stays_unique = ed_stays_sorted.drop_duplicates(subset='hadm_id', keep='first')

# Create the lookup dictionary from the unique entries
stay_id_lookup = ed_stays_unique.set_index('hadm_id')['stay_id'].to_dict()

# Fill NaNs in ecg_dataset['ed_stay_id'] using the lookup dictionary
ecg_dataset['ed_stay_id'] = ecg_dataset['ed_stay_id'].fillna(ecg_dataset['hadm_id'].map(stay_id_lookup))

In [124]:
ecg_dataset['ed_stay_id'].isnull().sum()

47975

In [125]:
ed_stays['hadm_id'].isna().sum()

222071

In [126]:
# Create the lookup dictionary from the unique entries for arrival transport
arrival_lookup = ed_stays_unique.set_index('hadm_id')['arrival_transport'].to_dict()

ecg_dataset['arrival_transport'] = ecg_dataset['arrival_transport'].fillna(ecg_dataset['hadm_id'].map(arrival_lookup))

In [127]:
# Define a regex pattern to capture variations of chest pain, dyspnea, and palpitations
pattern = r'(chest[\s-]*pain|dyspnea|dysnea|shortness[\s-]*of[\s-]*breath|palpitation|palpitations|chest[\s-]*tightness|angina pectoris)'

stay_ids = triage[triage['chiefcomplaint'].str.contains(pattern, case=False, na=False, regex=True)]['stay_id'].unique()
ecg_dataset['suggestive_symptoms'] = ecg_dataset['ed_stay_id'].isin(stay_ids).astype(int)

/tmp/ipykernel_3335723/189721783.py:4: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  stay_ids = triage[triage['chiefcomplaint'].str.contains(pattern, case=False, na=False, regex=True)]['stay_id'].unique()


In [128]:
ecg_dataset['suggestive_symptoms'].value_counts()

suggestive_symptoms
0    141171
1     39515
Name: count, dtype: int64

In [129]:
# Define a regex pattern to capture variations of chest pain, dyspnea, and palpitations
pattern = r'(chest[\s-]*pain|chest[\s-]*tightness|angina pectoris)'

stay_ids = triage[triage['chiefcomplaint'].str.contains(pattern, case=False, na=False, regex=True)]['stay_id'].unique()
ecg_dataset['chest_pain'] = ecg_dataset['ed_stay_id'].isin(stay_ids).astype(int)

/tmp/ipykernel_3335723/1226611434.py:4: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  stay_ids = triage[triage['chiefcomplaint'].str.contains(pattern, case=False, na=False, regex=True)]['stay_id'].unique()


In [130]:
ecg_dataset['chest_pain'].value_counts()

chest_pain
0    157212
1     23474
Name: count, dtype: int64

In [131]:
# Define a regex pattern to capture variations of chest pain, dyspnea, and palpitations
pattern = r'(dyspnea|dysnea|shortness[\s-]*of[\s-]*breath)'

stay_ids = triage[triage['chiefcomplaint'].str.contains(pattern, case=False, na=False, regex=True)]['stay_id'].unique()
ecg_dataset['dyspnea'] = ecg_dataset['ed_stay_id'].isin(stay_ids).astype(int)

/tmp/ipykernel_3335723/3204801630.py:4: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  stay_ids = triage[triage['chiefcomplaint'].str.contains(pattern, case=False, na=False, regex=True)]['stay_id'].unique()


In [132]:
ecg_dataset['dyspnea'].value_counts()

dyspnea
0    164727
1     15959
Name: count, dtype: int64

In [133]:
# Define a regex pattern to capture variations of chest pain, dyspnea, and palpitations
pattern = r'(palpitation|palpitations)'

stay_ids = triage[triage['chiefcomplaint'].str.contains(pattern, case=False, na=False, regex=True)]['stay_id'].unique()
ecg_dataset['palpitations'] = ecg_dataset['ed_stay_id'].isin(stay_ids).astype(int)

/tmp/ipykernel_3335723/2530939064.py:4: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  stay_ids = triage[triage['chiefcomplaint'].str.contains(pattern, case=False, na=False, regex=True)]['stay_id'].unique()


In [134]:
ecg_dataset['palpitations'].value_counts()

palpitations
0    177131
1      3555
Name: count, dtype: int64

In [135]:
# Initialize the new columns with 0
ecg_dataset['race_white'] = 0
ecg_dataset['race_black'] = 0
ecg_dataset['race_hispanic'] = 0
ecg_dataset['race_asian'] = 0
ecg_dataset['race_other'] = 0

# Fill in the columns based on conditions
ecg_dataset['race_white'] = ecg_dataset['race'].str.contains('WHITE', case=False, na=False).astype(int)
ecg_dataset['race_black'] = ecg_dataset['race'].str.contains('BLACK', case=False, na=False).astype(int)
ecg_dataset['race_hispanic'] = ecg_dataset['race'].str.contains('HISPANIC|LATINO', case=False, na=False, regex=True).astype(int)
ecg_dataset['race_asian'] = ecg_dataset['race'].str.contains('ASIAN', case=False, na=False).astype(int)

# For race_other, it should be 1 where none of the above conditions are met
ecg_dataset['race_other'] = (
    (ecg_dataset['race_white'] == 0) &
    (ecg_dataset['race_black'] == 0) &
    (ecg_dataset['race_hispanic'] == 0) &
    (ecg_dataset['race_asian'] == 0)
).astype(int)

In [136]:
# Drop the 'race' column from ecg_dataset
ecg_dataset.drop(columns=['race'], inplace=True)

In [137]:
# Initialize the new columns with 0
ecg_dataset['arrival_walk_in'] = 0
ecg_dataset['arrival_ambulance'] = 0
ecg_dataset['arrival_unknown'] = 0

# Fill in the columns based on conditions
ecg_dataset['arrival_walk_in'] = ecg_dataset['arrival_transport'].str.contains('WALK IN', case=False, na=False).astype(int)
ecg_dataset['arrival_ambulance'] = ecg_dataset['arrival_transport'].str.contains('AMBULANCE|HELICOPTER', case=False, na=False, regex=True).astype(int)

# Set arrival_unknown only if neither arrival_walk_in nor arrival_ambulance is set
ecg_dataset['arrival_unknown'] = (
    (~ecg_dataset['arrival_walk_in'].astype(bool)) & 
    (~ecg_dataset['arrival_ambulance'].astype(bool))
).astype(int)

In [138]:
ecg_dataset.drop(columns=['arrival_transport'], inplace=True)

In [139]:
# Specify the file path where you want to save the CSV file
file_path = os.path.join(path_to_local_files, 'mimic_ecg_dataset.csv')

# Save the dataframe to a CSV file
ecg_dataset.to_csv(file_path, index=False)